In [1]:
try:
    import obspy
    obspy.read()
except TypeError:
    # Needs to restart the runtime once, because obspy only works properly after restart.
    print('Stopping RUNTIME. If you run this code for the first time, this is expected. Colaboratory will restart automatically. Please run again.')
    exit()

In [2]:
import obspy
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from pyproj import CRS, Transformer
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import torch

import seisbench.models as sbm

sns.set(font_scale=1.2)
sns.set_style("ticks")

In [3]:
# Projections
wgs84 = CRS.from_epsg(4326)
local_crs = CRS.from_epsg(9155)  # SIRGAS-Chile 2016 / UTM zone 19S
transformer = Transformer.from_crs(wgs84, local_crs)

In [4]:
client = Client("GFZ")

t0 = UTCDateTime("2014/05/01 00:00:00")
t1 = t0 + 1 * 60 * 60
# t1 = t0 + 24 * 60 * 60   # Full day, requires more memory
stream = client.get_waveforms(network="CX", station="*", location="*", channel="HH?", starttime=t0, endtime=t1)

inv = client.get_stations(network="CX", station="*", location="*", channel="HH?", starttime=t0, endtime=t1)

In [5]:
picker = sbm.PhaseNet.from_pretrained("instance")

r=1
# if torch.cuda.is_available():
#     picker.cuda()

# We tuned the thresholds a bit - Feel free to play around with these values
picks = picker.classify(stream, batch_size=256, P_threshold=0.075*r, S_threshold=0.1*r).picks

Counter([p.phase for p in picks])  # Output number of P and S picks

Counter({'P': 498, 'S': 266})

In [7]:
pick_df = []
for p in picks:
    pick_df.append({
        "id": p.trace_id,
        "timestamp": p.peak_time.datetime,
        "prob": p.peak_value,
        "type": p.phase.lower()
    })
pick_df = pd.DataFrame(pick_df)

station_df = []
for station in inv[0]:
    station_df.append({
        "id": f"CX.{station.code}.",
        "longitude": station.longitude,
        "latitude": station.latitude,
        "elevation(m)": station.elevation
    })
station_df = pd.DataFrame(station_df)

station_df["x(km)"] = station_df.apply(lambda x: transformer.transform(x["latitude"], x["longitude"])[0] / 1e3, axis=1)
station_df["y(km)"] = station_df.apply(lambda x: transformer.transform(x["latitude"], x["longitude"])[1] / 1e3, axis=1)
station_df["z(km)"] = - station_df["elevation(m)"] / 1e3

In [8]:
config={}
config["x(km)"] = (250, 600)
config["y(km)"] = (7200, 8000)
config["vel"] = {"P": 7.0, "S": 7.0 / 1.75}  # We assume rather high velocities as we expect deeper 
config["z(km)"] = (0, 150)
config["ncpu"] = 2
config['P_phase']=True
config['S_phase']=True

config['epoch_before_decay']=500
config['epoch_after_decay']=500

In [9]:
import pickle

with open('pick_df.pkl', 'wb') as f:
    pickle.dump(pick_df, f)
    
with open('station_df.pkl', 'wb') as f:
    pickle.dump(station_df, f)
with open('config.pkl', 'wb') as f:
    pickle.dump(config, f)


In [1]:
import pickle

# Load pick_df
with open('pick_df.pkl', 'rb') as f:
    pick_df = pickle.load(f)

# Load station_df
with open('station_df.pkl', 'rb') as f:
    station_df = pickle.load(f)

# Load config
with open('config.pkl', 'rb') as f:
    config = pickle.load(f)

In [2]:
# import os
# import sys
# current_dir = os.getcwd()
# parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

# if parent_dir not in sys.path:
#     sys.path.append(parent_dir)
from harpa import association
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="This overload of addcmul_ is deprecated")
pick_df_out, catalog_df=association(pick_df,station_df,config,verbose=100)

Associating 764 picks separated into 24 slides with 2 CPUs


  0%|          | 0/25 [00:00<?, ?it/s]

Pick format: Pick format:    station_id               timestamp type  time_ref
6   CX.MNMCX. 2014-05-01 00:12:47.630    S    708.26
14  CX.MNMCX. 2014-05-01 00:30:59.030    S   1799.66
Station format:   station_id               timestamp type  time_ref
0  CX.MNMCX. 2014-05-01 00:00:13.850    P     11.14
1  CX.MNMCX. 2014-05-01 00:00:27.980    P     25.27
Station format:   station_id       x(km)        y(km)  z(km)  \
0   CX.PB01.  449.359240  7672.990624 -0.900   
1   CX.PB02.  407.073618  7642.202105 -1.015   

                                 arrival_time_list_P  \
0  [71.39, 138.26, 1515.29, 1989.2, 2130.03, 2985...   
1                   [340.5, 358.39, 1520.3, 2132.49]   

              arrival_pickindex_list_P                  arrival_time_list_S  \
0  [311, 312, 323, 329, 330, 350, 351]   [24.36, 1996.99, 2371.58, 3003.63]   
1                 [359, 360, 369, 372]  [1231.41, 1998.51, 2378.15, 3009.8]   

  arrival_pickindex_list_S  
0         [41, 56, 60, 65]  
1         [78, 84

100%|██████████| 25/25 [00:00<00:00, 208.15it/s]

removed 0 repeated events, left 1 unique events
 Associated 1 unique events


In [45]:
!export NUMBA_NUM_THREADS=40

In [40]:
pick_df

,id,timestamp,prob,type,labels
0,CX.MNMCX.,2014-05-01 00:00:13.850,0.805685,p,0
1,CX.MNMCX.,2014-05-01 00:00:27.980,0.351833,p,0
2,CX.MNMCX.,2014-05-01 00:00:39.360,0.106077,s,0
3,CX.MNMCX.,2014-05-01 00:04:27.210,0.617188,s,1
4,CX.MNMCX.,2014-05-01 00:08:11.290,0.628565,s,2
...,...,...,...,...,...
759,CX.PSGCX.,2014-05-01 00:52:29.310,0.987283,p,10
760,CX.PSGCX.,2014-05-01 00:56:50.840,0.509280,p,11
761,CX.PSGCX.,2014-05-01 00:59:09.450,0.784912,p,17
762,CX.PSGCX.,2014-05-01 00:59:34.480,0.686860,p,17
